# Experiment Design: Algorithmic Order Execution with Q-Learning

## Overview
This experiment implements a simplified algorithmic order execution environment using the ABIDES-Gym framework, integrated with a Q-learning reinforcement learning agent. The goal is to evaluate four execution strategies (TWAP, VWAP, IS, and POV) in a simulated market environment, focusing on minimizing transaction costs and market impact while executing a parent order. The design prioritizes quick execution on a CPU, enabling visualization of results with minimal computational resources.

## Environment Design
- **Base Framework**: Built on `AbidesGymMarketsEnv`, a Gym-compatible environment from ABIDES-Gym, which simulates a market with background noise agents.
- **Market Settings**:
  - **Market Open and Close**: Starts at 09:30:00 and closes at 09:35:00, limiting the simulation to a 5-minute window.
  - **First Interval**: 10 seconds (`00:00:10`), allowing the agent to begin trading shortly after market open.
  - **Execution Window**: 1 minute (`00:01:00`), providing a short period to execute the parent order.
  - **Timestep Duration**: 5 seconds (`5s`), resulting in approximately 12 steps per episode for fine-grained control.
- **Agent Parameters**:
  - **Starting Cash**: 10,000 units, a modest amount to reduce computational load.
  - **Parent Order Size**: 500 shares, a manageable order volume.
  - **Max Order Size**: 100 shares per trade, limiting individual order sizes.
  - **Noise Agents**: 5 background agents to provide liquidity and market activity.

## State Space
- **Definition**: The state space is a critical component of the reinforcement learning environment, representing the agent's perception of the market at any given time. It includes 8 core features:
  - **holdings_pct**: The percentage of the parent order size that has been executed or held, calculated as the current holdings divided by the parent order size. This indicates the progress of order execution relative to the target.
  - **time_pct**: The fraction of the execution window that has elapsed, computed as the time since the parent order arrival divided by the total execution window. This helps the agent manage timing constraints.
  - **diff_pct**: The difference between holdings_pct and time_pct, reflecting whether the agent is ahead or behind the expected execution pace. A positive value suggests over-execution, while a negative value indicates under-execution.
  - **imbalance_all**: The market imbalance across all depth levels, derived from the difference between buy and sell orders. This provides insight into market pressure and potential price movements.
  - **imbalance_5**: The market imbalance at the top 5 levels of the order book, offering a more focused view of immediate liquidity and order book dynamics.
  - **price_impact**: The logarithmic ratio of the current mid-price to the entry price (for BUY) or vice versa (for SELL), capturing the price change due to the agent's trades. This measures market impact and execution cost.
  - **spread**: The difference between the best ask and best bid prices, indicating market liquidity and transaction cost potential.
  - **direction_feature**: The difference between the mid-price and the last transaction price, providing a directional signal for price trends.
- **Padded Returns**: In addition to these 8 features, the state includes padded returns calculated from the difference in mid-prices over the state history length. With a history length of 4, this adds 3 padded return values (since `np.diff` on 4 prices yields 3 differences), bringing the total to 11 features. Padded returns are initialized with zeros and updated with actual returns, enabling the agent to consider price momentum.


## Action Space
- **Definition**: The action space is discrete, consisting of 3 actions:
  - **MKT (Market Order)**: Executes an order immediately at the current market price, with a size determined by the strategy-specific function.
  - **LMT (Limit Order)**: Places an order at a specified limit price (set to the near touch), with a size based on the strategy, waiting for market conditions to fill it.
  - **Hold**: Takes no action, allowing the agent to wait and reassess the state in the next timestep.
- **Mapping to ABIDES**: These actions are mapped to ABIDES simulator orders via the `_map_action_space_to_ABIDES_SIMULATOR_SPACE` method, where MKT and LMT orders include strategy-specific sizes, and Hold returns an empty list to skip trading.


## Reward Function
- **Definition**: The reward function is based on the profit and loss (PNL) normalized by the parent order size, with additional penalties for over- or under-execution:
  - **PNL Calculation**: Computed in `raw_state_to_reward` as the sum of (entry_price - fill_price) * quantity for BUY orders, or (fill_price - entry_price) * quantity for SELL orders, using data from `inter_wakeup_executed_orders`. This reflects the financial outcome of trades.
  - **Normalization**: The PNL is divided by the parent order size to standardize the reward, making it independent of order magnitude and comparable across episodes.
  - **Penalties**: In `raw_state_to_update_reward`, penalties are applied at episode end:
    - **Too Much Reward Update (-100)**: Applied if holdings exceed the parent order size, penalizing over-execution.
    - **Not Enough Reward Update (-1000)**: Applied if holdings are below the target, penalizing under-execution.
    - **Just Quantity Reward Update (0)**: Default reward if execution matches the parent order size exactly.

## Agent Design
- **Q-Learning Agent**: Implements a basic Q-learning algorithm with:
  - **State Discretization**: Uses 3 bins per feature (holdings_pct, time_pct, diff_pct) for 27 states, ensuring manageable state space.
  - **Action Selection**: Epsilon-greedy policy with 0.1 exploration rate.
  - **Learning Parameters**: Learning rate of 0.1 and discount factor of 0.9.
- **Strategy Implementation**: 
  - **TWAP**: Uniform time-based order placement.
  - **VWAP**: Volume-weighted average price with a linear approximation.
  - **IS**: Implementation shortfall with half max order size.
  - **POV**: Percentage of volume with 10% of current volume.
  - Order sizes are dynamically calculated based on strategy rules and current state.

## Simulation Design
- **Episodes**: 5 iterations per strategy to ensure quick completion.
- **Timeout**: 300 seconds (5 minutes) per run to prevent hangs.
- **Metrics Tracked**: Rewards, execution costs, trade schedules, and market volumes for each episode.
- **Visualization**: Three plots generated:
  - **Reward Curves**: Tracks reward progression per episode.
  - **Execution Cost Comparison**: Boxplot of normalized costs across strategies.
  - **Trade Schedule and Market Volume**: Plots cumulative executed quantity and scaled volume over time for each strategy.

## Objectives
- Demonstrate the performance of four execution strategies in a controlled, lightweight environment.
- Provide visual feedback (plots) to compare strategy effectiveness, focusing on trade scheduling.
- Ensure compatibility with CPU-only systems by minimizing computational load.

## Assumptions and Limitations
- Assumes sufficient market liquidity from noise agents for trade execution.
- Simplifies VWAP with a linear volume profile due to lack of real-time volume data.
- Short time period and small order sizes may not fully reflect real-world dynamics but are optimized for demonstration.

In [5]:
import importlib  # Import module for dynamic importing of configurations
from dataclasses import asdict, dataclass, field  # Import dataclass utilities for object creation
from typing import Any, Dict, List, Tuple  # Import type hints for type checking
from abc import ABC  # Import ABC for abstract base class definition

import gym  # Import Gym library for reinforcement learning environment
import numpy as np  # Import NumPy for numerical operations

import abides_markets.agents.utils as markets_agent_utils  # Import ABIDES market utilities
from abides_core import NanosecondTime  # Import NanosecondTime type for time handling
from abides_core.utils import str_to_ns  # Import function to convert time strings to nanoseconds
from abides_core.generators import ConstantTimeGenerator  # Import time generator for wakeups
from abides_gym.envs.markets_environment import AbidesGymMarketsEnv  # Import base environment class
import logging  # Import logging module for debug output
import time  # Import time module for timing operations

# Configure logging with debug level and a specific format
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)  # Create a logger instance with the module name

class QLearningAgent:
    """A Q-learning agent for reinforcement learning in the market environment.

    Parameters
    ----------
    action_space : gym.Space
        The action space defining possible actions (MKT, LMT, Hold).
    strategy : str
        The execution strategy (TWAP, VWAP, IS, or POV).

    Attributes
    ----------
    q_table : ndarray
        A 27x3 array storing Q-values for 27 states and 3 actions.
    action_space : gym.Space
        The action space for the agent.
    learning_rate : float
        The learning rate for Q-value updates (0.1).
    discount_factor : float
        The discount factor for future rewards (0.9).
    epsilon : float
        The exploration rate for epsilon-greedy policy (0.01, reduced from 0.1).
    strategy : str
        The selected execution strategy.
    """

    def __init__(self, action_space, strategy: str):
        # Initialize Q-table with zeros for 27 states and 3 actions
        self.q_table = np.zeros((27, 3))
        # Assign the action space to the instance
        self.action_space = action_space
        # Set learning rate
        self.learning_rate = 0.1
        # Set discount factor
        self.discount_factor = 0.9
        # Set reduced exploration rate to minimize deviation from strategy
        self.epsilon = 0.01
        # Store the selected strategy
        self.strategy = strategy

    def discretize_state(self, state):
        """Discretize the continuous state into a discrete index.

        Parameters
        ----------
        state : ndarray
            The continuous state vector (holdings_pct, time_pct, diff_pct).

        Returns
        -------
        int
            The discrete state index (0 to 26).

        Notes
        -----
        Uses three bins per feature (-inf, -0.5), [-0.5, 0.5), [0.5, inf) to create 27 states.
        """
        # Define bin edges for three levels per feature
        bins = np.array([-0.5, 0.5])
        # Convert state values into bin indices using digitize
        indices = np.digitize([state[0], state[1], state[2]], bins)
        # Ensure indices are within the valid range [0, 2]
        indices = np.clip(indices, 0, 2)
        # Compute a unique state index from a 3D grid (3^3 = 27 states)
        state_idx = indices[0] * 9 + indices[1] * 3 + indices[2]
        # Cap the state index to the maximum valid value
        state_idx = min(state_idx, 26)
        # Log debug information about state discretization
        logger.debug(f"State: {state[:3]}, Indices: {indices}, State_idx: {state_idx}")
        # Return the discrete state index
        return state_idx

    def _get_strategy_order_size(self, env):
        """Calculate the order size based on the selected strategy.

        Parameters
        ----------
        env : SubGymMarketsExecutionEnv_v0
            The environment instance providing tracker data.

        Returns
        -------
        int
            The calculated order size, bounded by max_order_size.

        Notes
        -----
        Implements TWAP, VWAP, IS, and POV strategies using time_pct, holdings_pct, executed_qty, and volume.
        """
        # Get the time percentage from the environment tracker
        time_pct = env.custom_metrics_tracker.time_pct
        # Get the holdings percentage from the environment tracker
        holdings_pct = env.custom_metrics_tracker.holdings_pct
        # Get the executed quantity from the environment tracker
        executed_qty = env.custom_metrics_tracker.executed_quantity
        # Get the volume from the environment tracker
        volume = env.custom_metrics_tracker.volume
        # Check if strategy attribute exists, otherwise default to TWAP
        if hasattr(self, 'strategy'):
            strategy = self.strategy
        else:
            strategy = 'TWAP'
        # Apply TWAP strategy: target quantity based on time percentage
        if strategy == "TWAP":
            target_qty = time_pct * env.parent_order_size
            return int(max(1, min(env.max_order_size, abs(target_qty - executed_qty))))
        # Apply VWAP strategy: target quantity based on step index
        elif strategy == "VWAP":
            step_idx = min(int(time_pct * (env.execution_window // env.timestep_duration)), 9)
            target_qty = (step_idx / 10) * env.parent_order_size
            return int(max(1, min(env.max_order_size, abs(target_qty - executed_qty))))
        # Apply IS strategy: half max order size
        elif strategy == "IS":
            return int(max(1, min(env.max_order_size / 2, env.parent_order_size - abs(executed_qty))))
        # Apply POV strategy: 10% of volume
        elif strategy == "POV":
            target_qty = 0.1 * volume
            return int(max(1, min(env.max_order_size, abs(target_qty - executed_qty))))
        # Default return if no strategy matches
        return 1

    def choose_action(self, state, env):
        """Select an action using an epsilon-greedy policy.

        Parameters
        ----------
        state : ndarray
            The current state vector.
        env : SubGymMarketsExecutionEnv_v0
            The environment instance (unused in this method).

        Returns
        -------
        int
            The chosen action index (0, 1, or 2).

        Notes
        -----
        Explores with probability epsilon, otherwise exploits the max Q-value.
        """
        # Generate a random number to decide exploration vs exploitation
        if np.random.random() < self.epsilon:
            return self.action_space.sample()
        # Discretize the current state to get its index
        state_idx = self.discretize_state(state)
        # Choose the action with the highest Q-value
        return np.argmax(self.q_table[state_idx])

    def learn(self, state, action, reward, next_state, env):
        """Update the Q-table based on the Q-learning rule.

        Parameters
        ----------
        state : ndarray
            The current state vector.
        action : int
            The action taken.
        reward : float
            The reward received.
        next_state : ndarray
            The next state vector.
        env : SubGymMarketsExecutionEnv_v0
            The environment instance (unused in this method).

        Notes
        -----
        Updates Q-value using the formula: Q(s,a) += lr * [r + df * max(Q(s',a')) - Q(s,a)].
        """
        # Discretize the current state
        state_idx = self.discretize_state(state)
        # Discretize the next state
        next_state_idx = self.discretize_state(next_state)
        # Get the old Q-value for the current state-action pair
        old_value = self.q_table[state_idx, action]
        # Get the maximum Q-value for the next state
        next_max = np.max(self.q_table[next_state_idx])
        # Calculate the new Q-value using the Q-learning update rule
        new_value = old_value + self.learning_rate * (reward + self.discount_factor * next_max - old_value)
        # Update the Q-table with the new value
        self.q_table[state_idx, action] = new_value

class SubGymMarketsExecutionEnv_v0(AbidesGymMarketsEnv):
    """A Gym environment for algorithmic order execution using ABIDES.

    Parameters
    ----------
    background_config : Any, optional
        The configuration module for background agents (default: "rmsc04").
    mkt_close : str, optional
        Market close time (default: "09:35:00").
    timestep_duration : str, optional
        Duration between agent wakeups (default: "5s").
    starting_cash : int, optional
        Initial cash for the agent (default: 10000).
    max_order_size : int, optional
        Maximum order size per trade (default: 100).
    state_history_length : int, optional
        Length of state history buffer (default: 4).
    market_data_buffer_length : int, optional
        Length of market data buffer (default: 5).
    first_interval : str, optional
        Initial delay before first wakeup (default: "00:00:10").
    parent_order_size : int, optional
        Total size of the parent order (default: 500).
    execution_window : str, optional
        Time window for order execution (default: "00:01:00").
    direction : str, optional
        Trade direction (default: "BUY").
    not_enough_reward_update : int, optional
        Penalty for under-execution (default: -500, adjusted from -1000).
    too_much_reward_update : int, optional
        Penalty for over-execution (default: -200, adjusted from -100).
    just_quantity_reward_update : int, optional
        Reward for exact execution (default: 0).
    debug_mode : bool, optional
        Enable debug logging (default: True).
    background_config_extra_kvargs : Dict[str, Any], optional
        Extra arguments for background config (default: {"num_noise_agents": 5}).

    Attributes
    ----------
    custom_metrics_tracker : CustomMetricsTracker
        Tracks metrics like rewards, quantities, and trade schedule.
    """

    # Decorator to ignore buffers in raw state processing
    raw_state_pre_process = markets_agent_utils.ignore_buffers_decorator
    # Decorator to ignore market data buffers in state processing
    raw_state_to_state_pre_process = markets_agent_utils.ignore_mkt_data_buffer_decorator

    @dataclass
    class CustomMetricsTracker(ABC):
        """Tracks custom metrics for the environment.

        Attributes
        ----------
        slippage_reward : float
            Reward from slippage (default: 0).
        late_penalty_reward : float
            Penalty for late execution (default: 0).
        executed_quantity : int
            Total executed quantity (default: 0).
        remaining_quantity : int
            Remaining quantity to execute (default: 0).
        action_counter : Dict[str, int]
            Counter for each action type (default: empty dict).
        holdings_pct : float
            Percentage of holdings (default: 0).
        time_pct : float
            Percentage of execution time elapsed (default: 0).
        diff_pct : float
            Difference between holdings and time percentage (default: 0).
        imbalance_all : float
            Market imbalance at all depths (default: 0).
        imbalance_5 : float
            Market imbalance at 5 levels (default: 0).
        price_impact : float
            Price impact of trades (default: 0).
        spread : float
            Bid-ask spread (default: 0).
        direction_feature : float
            Direction feature (default: 0).
        num_max_steps_per_episode : float
            Maximum steps per episode (default: 0).
        trade_schedule : List[Tuple[float, float]]
            Schedule of (time_pct, executed_quantity) pairs (default: empty list).
        """
        slippage_reward: float = 0
        late_penalty_reward: float = 0
        executed_quantity: int = 0
        remaining_quantity: int = 0
        action_counter: Dict[str, int] = field(default_factory=dict)
        holdings_pct: float = 0
        time_pct: float = 0
        diff_pct: float = 0
        imbalance_all: float = 0
        imbalance_5: float = 0
        price_impact: float = 0
        spread: float = 0
        direction_feature: float = 0
        num_max_steps_per_episode: float = 0
        trade_schedule: List[Tuple[float, float]] = field(default_factory=list)

    def __init__(
        self,
        background_config: Any = "rmsc04",
        mkt_close: str = "09:35:00",
        timestep_duration: str = "5s",
        starting_cash: int = 10000,
        max_order_size: int = 100,
        state_history_length: int = 4,
        market_data_buffer_length: int = 5,
        first_interval: str = "00:00:10",
        parent_order_size: int = 500,
        execution_window: str = "00:01:00",
        direction: str = "BUY",
        not_enough_reward_update: int = -500,  # Adjusted penalty for under-execution
        too_much_reward_update: int = -200,  # Adjusted penalty for over-execution
        just_quantity_reward_update: int = 0,
        debug_mode: bool = True,
        background_config_extra_kvargs: Dict[str, Any] = {"num_noise_agents": 5},
    ) -> None:
        # Dynamically import the background configuration module
        self.background_config = importlib.import_module(
            "abides_markets.configs.{}".format(background_config), package=None
        )
        # Convert market close time to nanoseconds
        self.mkt_close = str_to_ns(mkt_close)
        # Convert timestep duration to nanoseconds
        self.timestep_duration = str_to_ns(timestep_duration)
        # Set initial cash amount
        self.starting_cash = starting_cash
        # Set maximum order size
        self.max_order_size = max_order_size
        # Set state history length
        self.state_history_length = state_history_length
        # Set market data buffer length
        self.market_data_buffer_length = market_data_buffer_length
        # Convert first interval to nanoseconds
        self.first_interval = str_to_ns(first_interval)
        # Set parent order size
        self.parent_order_size = parent_order_size
        # Convert execution window to nanoseconds
        self.execution_window = str_to_ns(execution_window)
        # Set trade direction
        self.direction = direction
        # Enable or disable debug mode
        self.debug_mode = debug_mode
        # Set penalty for too much execution
        self.too_much_reward_update = too_much_reward_update
        # Set penalty for not enough execution
        self.not_enough_reward_update = not_enough_reward_update
        # Set reward for exact execution
        self.just_quantity_reward_update = just_quantity_reward_update
        # Initialize entry price
        self.entry_price = 0.0
        # Initialize near touch price
        self.near_touch = 0.0
        # Initialize step index
        self.step_index = 0
        # Create instance of CustomMetricsTracker
        self.custom_metrics_tracker = self.CustomMetricsTracker()
        # Assert valid background configuration
        assert background_config in ["rmsc03", "rmsc04", "smc_01"], "Invalid background config"
        # Assert valid first interval range
        assert (self.first_interval <= str_to_ns("16:00:00")) & (self.first_interval >= str_to_ns("00:00:00")), "Invalid first_interval"
        # Assert valid market close range
        assert (self.mkt_close <= str_to_ns("16:00:00")) & (self.mkt_close >= str_to_ns("09:30:00")), "Invalid mkt_close"
        # Assert valid timestep duration range
        assert (self.timestep_duration <= str_to_ns("06:30:00")) & (self.timestep_duration >= str_to_ns("00:00:00")), "Invalid timestep_duration"
        # Assert valid starting cash
        assert (type(self.starting_cash) == int) & (self.starting_cash >= 0), "Invalid starting_cash"
        # Assert valid max order size
        assert (type(self.max_order_size) == int) & (self.max_order_size >= 0), "Invalid max_order_size"
        # Assert valid state history length
        assert (type(self.state_history_length) == int) & (self.state_history_length >= 0), "Invalid state_history_length"
        # Assert valid market data buffer length
        assert (type(self.market_data_buffer_length) == int) & (self.market_data_buffer_length >= 0), "Invalid market_data_buffer_length"
        # Assert valid debug mode
        assert self.debug_mode in [True, False], "Invalid debug_mode"
        # Assert valid direction
        assert self.direction in ["BUY", "SELL"], "Invalid direction"
        # Assert valid parent order size
        assert (type(self.parent_order_size) == int) & (self.parent_order_size >= 0), "Invalid parent_order_size"
        # Assert valid execution window
        assert (self.execution_window <= str_to_ns("06:30:00")) & (self.execution_window >= str_to_ns("00:00:00")), "Invalid execution_window"
        # Prepare background configuration arguments
        background_config_args = {"end_time": self.mkt_close}
        # Update background config with extra arguments
        background_config_args.update(background_config_extra_kvargs)
        # Initialize the parent AbidesGymMarketsEnv
        super().__init__(
            background_config_pair=(self.background_config.build_config, background_config_args),
            wakeup_interval_generator=ConstantTimeGenerator(step_duration=self.timestep_duration),
            starting_cash=self.starting_cash,
            state_buffer_length=self.state_history_length,
            market_data_buffer_length=self.market_data_buffer_length,
            first_interval=self.first_interval,
        )
        # Set number of actions
        self.num_actions = 3
        # Define the action space as discrete with 3 actions
        self.action_space = gym.spaces.Discrete(self.num_actions)
        # Initialize action counters for each action
        for i in range(self.num_actions):
            self.custom_metrics_tracker.action_counter[f"action_{i}"] = 0
        # Calculate total nanoseconds for the episode
        num_ns_episode = self.first_interval + self.execution_window
        # Set step length
        step_length = self.timestep_duration
        # Calculate maximum steps per episode
        num_max_steps_per_episode = num_ns_episode / step_length
        # Log the number of maximum steps
        logger.debug(f"Num max steps per episode: {num_max_steps_per_episode}")
        # Set the maximum steps in the tracker
        self.custom_metrics_tracker.num_max_steps_per_episode = num_max_steps_per_episode
        # Calculate total number of state features
        self.num_state_features = 8 + self.state_history_length - 1
        # Define upper bounds for state features
        self.state_highs = np.array([2, 2, 4, 1, 1, np.finfo(np.float32).max, np.finfo(np.float32).max, np.finfo(np.float32).max] + (self.state_history_length - 1) * [np.finfo(np.float32).max], dtype=np.float32).reshape(self.num_state_features, 1)
        # Define lower bounds for state features
        self.state_lows = np.array([-2, -2, -4, 0, 0, np.finfo(np.float32).min, np.finfo(np.float32).min, np.finfo(np.float32).min] + (self.state_history_length - 1) * [np.finfo(np.float32).min], dtype=np.float32).reshape(self.num_state_features, 1)
        # Define the observation space
        self.observation_space = gym.spaces.Box(self.state_lows, self.state_highs, shape=(self.num_state_features, 1), dtype=np.float32)
        # Set initial marked-to-market value
        self.previous_marked_to_market = self.starting_cash

    def _map_action_space_to_ABIDES_SIMULATOR_SPACE(self, action: int) -> List[Dict[str, Any]]:
        """Map Gym action to ABIDES simulator orders.

        Parameters
        ----------
        action : int
            The action index (0: MKT, 1: LMT, 2: Hold).

        Returns
        -------
        List[Dict[str, Any]]
            List of order dictionaries for the simulator.

        Raises
        ------
        ValueError
            If action is not in [0, 1, 2].
        """
        # Increment the counter for the chosen action
        self.custom_metrics_tracker.action_counter[f"action_{action}"] += 1
        # Handle Market (MKT) order
        if action == 0:
            order_size = self._get_strategy_order_size()
            return [{"type": "CCL_ALL"}, {"type": "MKT", "direction": self.direction, "size": order_size}]
        # Handle Limit (LMT) order
        elif action == 1:
            order_size = self._get_strategy_order_size()
            return [{"type": "CCL_ALL"}, {"type": "LMT", "direction": self.direction, "size": order_size, "limit_price": self.near_touch}]
        # Handle Hold action
        elif action == 2:
            return []
        # Raise error for invalid action
        else:
            raise ValueError(f"Action {action} is not supported")

    def _get_strategy_order_size(self) -> int:
        """Calculate the order size based on the selected strategy.

        Returns
        -------
        int
            The calculated order size, bounded by max_order_size.

        Notes
        -----
        Uses time_pct, holdings_pct, executed_qty, and volume from the tracker.
        """
        # Get the time percentage from the tracker
        time_pct = self.custom_metrics_tracker.time_pct
        # Get the holdings percentage from the tracker
        holdings_pct = self.custom_metrics_tracker.holdings_pct
        # Get the executed quantity from the tracker
        executed_qty = self.custom_metrics_tracker.executed_quantity
        # Get the volume from the tracker
        volume = self.custom_metrics_tracker.volume
        # Check if strategy attribute exists, otherwise default to TWAP
        if hasattr(self, 'strategy'):
            strategy = self.strategy
        else:
            strategy = 'TWAP'
        # Apply TWAP strategy: target quantity based on time percentage
        if strategy == "TWAP":
            target_qty = time_pct * self.parent_order_size
            return int(max(1, min(self.max_order_size, abs(target_qty - executed_qty))))
        # Apply VWAP strategy: target quantity based on step index
        elif strategy == "VWAP":
            step_idx = min(int(time_pct * (self.execution_window // self.timestep_duration)), 9)
            target_qty = (step_idx / 10) * self.parent_order_size
            return int(max(1, min(self.max_order_size, abs(target_qty - executed_qty))))
        # Apply IS strategy: half max order size
        elif strategy == "IS":
            return int(max(1, min(self.max_order_size / 2, self.parent_order_size - abs(executed_qty))))
        # Apply POV strategy: 10% of volume
        elif strategy == "POV":
            target_qty = 0.1 * volume
            return int(max(1, min(self.max_order_size, abs(target_qty - executed_qty))))
        # Default return if no strategy matches
        return 1

    @raw_state_to_state_pre_process
    def raw_state_to_state(self, raw_state: Dict[str, Any]) -> np.ndarray:
        """Convert raw state to a processed state vector.

        Parameters
        ----------
        raw_state : Dict[str, Any]
            Raw simulation data from the environment.

        Returns
        -------
        ndarray
            Processed state vector of shape (num_state_features, 1).

        Notes
        -----
        Computes features like holdings_pct, time_pct, and market imbalances, with safety checks for empty deques and type handling.
        """
        # Log the keys of the raw state for debugging
        logger.debug(f"Raw state keys: {raw_state.keys()}")
        # Extract bid data from raw state, handle empty deque or non-iterable
        bids = raw_state["parsed_mkt_data"]["bids"][-1] if raw_state["parsed_mkt_data"]["bids"] and len(raw_state["parsed_mkt_data"]["bids"]) > 0 else [[]]
        # Extract ask data from raw state, handle empty deque or non-iterable
        asks = raw_state["parsed_mkt_data"]["asks"][-1] if raw_state["parsed_mkt_data"]["asks"] and len(raw_state["parsed_mkt_data"]["asks"]) > 0 else [[]]
        # Extract last transaction data from raw state, handle empty deque or non-iterable
        last_transactions = raw_state["parsed_mkt_data"]["last_transaction"][-1] if raw_state["parsed_mkt_data"]["last_transaction"] and len(raw_state["parsed_mkt_data"]["last_transaction"]) > 0 else [0]
        # Get holdings data, converting to list if not already
        holdings = raw_state["internal_data"]["holdings"] if isinstance(raw_state["internal_data"]["holdings"], (list, np.ndarray)) else [raw_state["internal_data"]["holdings"]]
        # Log holdings type and value for debugging
        logger.debug(f"Holdings type: {type(raw_state['internal_data']['holdings'])}, value: {holdings}")
        # Calculate holdings percentage, default to 0 if parent_order_size is 0
        holdings_pct = holdings[-1] / self.parent_order_size if self.parent_order_size > 0 else 0.0
        # Get market open time, using last element if list
        mkt_open = raw_state["internal_data"]["mkt_open"][-1] if isinstance(raw_state["internal_data"]["mkt_open"], (list, np.ndarray)) else raw_state["internal_data"]["mkt_open"]
        # Get current time, using last element if list
        current_time = raw_state["internal_data"]["current_time"][-1] if isinstance(raw_state["internal_data"]["current_time"], (list, np.ndarray)) else raw_state["internal_data"]["current_time"]
        # Calculate time elapsed since parent order arrival
        time_from_parent_arrival = float(current_time) - float(mkt_open) - float(self.first_interval)
        # Calculate time percentage within execution window
        time_pct = time_from_parent_arrival / float(self.execution_window) if self.execution_window > 0 else 0.0
        # Calculate difference percentage
        diff_pct = holdings_pct - time_pct
        # Compute imbalances for all depths, handle empty bids/asks
        imbalances_all = [markets_agent_utils.get_imbalance(b, a, depth=None) for b, a in zip(bids if bids and isinstance(bids, list) else [[]], asks if asks and isinstance(asks, list) else [[]])]
        # Get the last imbalance value, default to 0 if empty
        imbalance_all = imbalances_all[-1] if imbalances_all else 0.0
        # Compute imbalances for 5 levels, handle empty bids/asks
        imbalances_5 = [markets_agent_utils.get_imbalance(b, a, depth=5) for b, a in zip(bids if bids and isinstance(bids, list) else [[]], asks if asks and isinstance(asks, list) else [[]])]
        # Get the last 5-level imbalance value, default to 0 if empty
        imbalance_5 = imbalances_5[-1] if imbalances_5 else 0.0
        # Compute mid prices from bids, asks, and last transactions, handle non-iterable cases
        mid_prices = []
        if bids and asks and last_transactions and isinstance(bids, list) and isinstance(asks, list) and isinstance(last_transactions, list):
            mid_prices = [markets_agent_utils.get_mid_price(b, a, lt) for b, a, lt in zip(bids, asks, last_transactions)]
        # Get the last mid price value, default to 0 if empty
        mid_price = mid_prices[-1] if mid_prices else 0.0
        # Set entry price on first step if mid_price is valid
        if self.step_index == 0 and mid_price > 0:
            self.entry_price = mid_price
        # Determine the best book price based on direction
        book = raw_state["parsed_mkt_data"]["bids"][-1] if raw_state["parsed_mkt_data"]["bids"] and len(raw_state["parsed_mkt_data"]["bids"]) > 0 and self.direction == "BUY" else raw_state["parsed_mkt_data"]["asks"][-1] if raw_state["parsed_mkt_data"]["asks"] and len(raw_state["parsed_mkt_data"]["asks"]) > 0 else []
        # Set near touch price from book or last transaction
        self.near_touch = book[0][0] if book and len(book) > 0 else last_transactions[-1] if last_transactions and len(last_transactions) > 0 else self.entry_price
        # Calculate price impact based on direction
        price_impact = np.log(mid_price / self.entry_price) if self.direction == "BUY" and self.entry_price > 0 else np.log(self.entry_price / mid_price) if self.entry_price > 0 else 0.0
        # Get best bid prices, handle empty bids
        best_bids = [bids[0][0] if len(bids) > 0 and isinstance(bids, list) else mid for bids, mid in zip([bids] if bids and isinstance(bids, list) else [[]], mid_prices)]
        # Get best ask prices, handle empty asks
        best_asks = [asks[0][0] if len(asks) > 0 and isinstance(asks, list) else mid for asks, mid in zip([asks] if asks and isinstance(asks, list) else [[]], mid_prices)]
        # Calculate spreads from best bids and asks
        spreads = np.array(best_asks) - np.array(best_bids) if len(best_bids) == len(best_asks) else np.array([0.0])
        # Get the last spread value
        spread = float(spreads[-1]) if spreads.size > 0 else 0.0
        # Calculate direction features
        direction_features = np.array(mid_prices) - np.array(last_transactions) if len(mid_prices) == len(last_transactions) else np.array([0.0])
        # Get the last direction feature value
        direction_feature = float(direction_features[-1]) if direction_features.size > 0 else 0.0
        # Calculate total volume from best bids and asks
        volume = sum(b[1] for b in bids[-1][:1]) + sum(a[1] for a in asks[-1][:1]) if bids and asks and bids[-1] and asks[-1] and isinstance(bids[-1], list) and isinstance(asks[-1], list) else 0.0
        # Calculate price returns
        returns = np.diff(mid_prices) if len(mid_prices) > 1 else np.array([0.0])
        # Initialize padded returns array
        padded_returns = np.zeros(self.state_history_length - 1)
        # Update padded returns with calculated returns
        padded_returns[-len(returns):] = returns if len(returns) > 0 else padded_returns
        # Update holdings percentage in tracker
        self.custom_metrics_tracker.holdings_pct = float(holdings_pct)
        # Update time percentage in tracker
        self.custom_metrics_tracker.time_pct = float(time_pct)
        # Update difference percentage in tracker
        self.custom_metrics_tracker.diff_pct = float(diff_pct)
        # Update all-depth imbalance in tracker
        self.custom_metrics_tracker.imbalance_all = float(imbalance_all)
        # Update 5-level imbalance in tracker
        self.custom_metrics_tracker.imbalance_5 = float(imbalance_5)
        # Update price impact in tracker
        self.custom_metrics_tracker.price_impact = float(price_impact)
        # Update spread in tracker
        self.custom_metrics_tracker.spread = float(spread)
        # Update direction feature in tracker
        self.custom_metrics_tracker.direction_feature = float(direction_feature)
        # Update volume in tracker
        self.custom_metrics_tracker.volume = float(volume)
        # Create computed state array
        computed_state = np.array([holdings_pct, time_pct, diff_pct, imbalance_all, imbalance_5, price_impact, spread, direction_feature] + padded_returns.tolist(), dtype=np.float32)
        # Increment step index
        self.step_index += 1
        # Reshape computed state to required dimensions
        return computed_state.reshape(self.num_state_features, 1)

    @raw_state_to_state_pre_process
    def raw_state_to_reward(self, raw_state: Dict[str, Any]) -> float:
        """Compute the reward based on trade execution.

        Parameters
        ----------
        raw_state : Dict[str, Any]
            Raw simulation data from the environment.

        Returns
        -------
        float
            The computed reward value.

        Notes
        -----
        Updates trade_schedule and calculates PNL based on executed orders, handling list-based order data with error checking.
        """
        # Get the entry price
        entry_price = self.entry_price
        # Get executed orders from raw state, expected as list of [fill_price, quantity]
        inter_wakeup_executed_orders = raw_state["internal_data"]["inter_wakeup_executed_orders"]
        # Log the structure of executed orders for debugging
        logger.debug(f"inter_wakeup_executed_orders: {inter_wakeup_executed_orders}")
        # Update trade schedule if there are executed orders
        if len(inter_wakeup_executed_orders) > 0:
            executed_qty = self.custom_metrics_tracker.executed_quantity
            if not any(t[1] == executed_qty for t in self.custom_metrics_tracker.trade_schedule):
                self.custom_metrics_tracker.trade_schedule.append((self.custom_metrics_tracker.time_pct, executed_qty))
                logger.debug(f"Updated trade_schedule: {self.custom_metrics_tracker.trade_schedule}")
        # Set PNL to 0 if no orders executed
        if len(inter_wakeup_executed_orders) == 0:
            pnl = 0.0
        # Calculate PNL based on direction, handling [fill_price, quantity] pairs with error checking
        else:
            pnl = 0.0
            for order in inter_wakeup_executed_orders:
                if len(order) >= 2:  # Ensure order has at least fill_price and quantity
                    fill_price = order[0]
                    quantity = order[1]
                    pnl += (entry_price - fill_price) * quantity if self.direction == "BUY" else (fill_price - entry_price) * quantity
                else:
                    logger.warning(f"Invalid order data: {order}, skipping contribution to PNL")
        # Store the PNL value
        self.pnl = float(pnl)
        # Update execution cost in tracker
        self.custom_metrics_tracker.execution_cost += abs(pnl) / self.parent_order_size if self.parent_order_size > 0 else 0.0
        # Calculate reward from PNL
        reward = pnl / self.parent_order_size if self.parent_order_size > 0 else 0.0
        # Update slippage reward in tracker
        self.custom_metrics_tracker.slippage_reward = float(reward)
        # Return the computed reward
        return reward

    @raw_state_pre_process
    def raw_state_to_update_reward(self, raw_state: Dict[str, Any]) -> float:
        """Compute the final reward update at episode end.

        Parameters
        ----------
        raw_state : Dict[str, Any]
            Raw simulation data from the environment.

        Returns
        -------
        float
            The update reward value.

        Notes
        -----
        Applies penalties or rewards based on execution quantity.
        """
        # Get holdings data, converting to list if not already
        holdings = raw_state["internal_data"]["holdings"] if isinstance(raw_state["internal_data"]["holdings"], (list, np.ndarray)) else [raw_state["internal_data"]["holdings"]]
        # Log holdings for debugging
        logger.debug(f"Update reward holdings: {holdings}")
        # Get parent order size
        parent_order_size = self.parent_order_size
        # Apply penalty for over-execution in BUY direction
        if (self.direction == "BUY") and (holdings[-1] >= parent_order_size):
            update_reward = abs(holdings[-1] - parent_order_size) * self.too_much_reward_update
        # Apply penalty for under-execution in BUY direction
        elif (self.direction == "BUY") and (holdings[-1] < parent_order_size):
            update_reward = abs(holdings[-1] - parent_order_size) * self.not_enough_reward_update
        # Apply penalty for over-execution in SELL direction
        elif (self.direction == "SELL") and (holdings[-1] <= -parent_order_size):
            update_reward = abs(holdings[-1] - parent_order_size) * self.too_much_reward_update
        # Apply penalty for under-execution in SELL direction
        elif (self.direction == "SELL") and (holdings[-1] > -parent_order_size):
            update_reward = abs(holdings[-1] - parent_order_size) * self.not_enough_reward_update
        # Apply default reward for exact execution
        else:
            update_reward = self.just_quantity_reward_update
        # Normalize update reward by parent order size
        update_reward = update_reward / parent_order_size if parent_order_size > 0 else 0.0
        # Update late penalty reward in tracker
        self.custom_metrics_tracker.late_penalty_reward = float(update_reward)
        # Return the update reward
        return update_reward

    @raw_state_pre_process
    def raw_state_to_done(self, raw_state: Dict[str, Any]) -> bool:
        """Determine if the episode is done.

        Parameters
        ----------
        raw_state : Dict[str, Any]
            Raw simulation data from the environment.

        Returns
        -------
        bool
            True if episode is done, False otherwise.

        Notes
        -----
        Checks for order completion or time limit.
        """
        # Get holdings data, converting to list if not already
        holdings = raw_state["internal_data"]["holdings"] if isinstance(raw_state["internal_data"]["holdings"], (list, np.ndarray)) else [raw_state["internal_data"]["holdings"]]
        # Log holdings for debugging
        logger.debug(f"Done holdings: {holdings}")
        # Get parent order size
        parent_order_size = self.parent_order_size
        # Get current time, using last element if list
        current_time = raw_state["internal_data"]["current_time"][-1] if isinstance(raw_state["internal_data"]["current_time"], (list, np.ndarray)) else raw_state["internal_data"]["current_time"]
        # Get market open time, using last element if list
        mkt_open = raw_state["internal_data"]["mkt_open"][-1] if isinstance(raw_state["internal_data"]["mkt_open"], (list, np.ndarray)) else raw_state["internal_data"]["mkt_open"]
        # Calculate time limit for the episode
        time_limit = float(mkt_open) + float(self.first_interval) + float(self.execution_window)
        # Determine if episode is done based on execution or time
        done = ((self.direction == "BUY" and holdings[-1] >= parent_order_size) or
                (self.direction == "SELL" and holdings[-1] <= -parent_order_size) or
                (float(current_time) >= time_limit))
        # Update executed quantity based on direction
        self.custom_metrics_tracker.executed_quantity = holdings[-1] if self.direction == "BUY" else -holdings[-1]
        # Calculate remaining quantity
        self.custom_metrics_tracker.remaining_quantity = parent_order_size - self.custom_metrics_tracker.executed_quantity
        # Return done status
        return done

    @raw_state_pre_process
    def raw_state_to_info(self, raw_state: Dict[str, Any]) -> Dict[str, Any]:
        """Generate info dictionary for debugging or logging.

        Parameters
        ----------
        raw_state : Dict[str, Any]
            Raw simulation data from the environment.

        Returns
        -------
        Dict[str, Any]
            Dictionary containing market and execution metrics.

        Notes
        -----
        Includes trade details and custom metrics if debug_mode is False.
        """
        # Get holdings data, converting to list if not already
        holdings = raw_state["internal_data"]["holdings"] if isinstance(raw_state["internal_data"]["holdings"], (list, np.ndarray)) else [raw_state["internal_data"]["holdings"]]
        # Log holdings for debugging
        logger.debug(f"Info holdings: {holdings}")
        # Get last transaction price
        last_transaction = raw_state["parsed_mkt_data"]["last_transaction"]
        # Get bid data
        bids = raw_state["parsed_mkt_data"]["bids"]
        # Get best bid price, default to last transaction if no bids
        best_bid = bids[0][0] if len(bids) > 0 else last_transaction
        # Get ask data
        asks = raw_state["parsed_mkt_data"]["asks"]
        # Get best ask price, default to last transaction if no asks
        best_ask = asks[0][0] if len(asks) > 0 else last_transaction
        # Get current time, using last element if list
        current_time = raw_state["internal_data"]["current_time"][-1] if isinstance(raw_state["internal_data"]["current_time"], (list, np.ndarray)) else raw_state["internal_data"]["current_time"]
        # Create info dictionary with market and execution metrics
        info = {
            "last_transaction": float(last_transaction),
            "best_bid": float(best_bid),
            "best_ask": float(best_ask),
            "current_time": float(current_time),
            "holdings": float(holdings[-1]),
            "parent_size": float(self.parent_order_size),
            "pnl": float(self.pnl),
            "reward": float(self.pnl / self.parent_order_size if self.parent_order_size > 0 else 0.0),
            "volume": self.custom_metrics_tracker.volume,
            "execution_cost": self.custom_metrics_tracker.execution_cost,
            "trade_schedule": self.custom_metrics_tracker.trade_schedule,
        }
        # Replace info with custom metrics if debug mode is off
        if not self.debug_mode:
            info = asdict(self.custom_metrics_tracker)
        # Return the info dictionary
        return info

def run_simulation(strategy: str, episodes: int = 5):
    """Run a simulation for a given strategy.

    Parameters
    ----------
    strategy : str
        The execution strategy (TWAP, VWAP, IS, or POV).
    episodes : int, optional
        Number of episodes to run (default: 5).

    Returns
    -------
    tuple
        (rewards, execution_costs, trade_schedules, market_volumes) for the simulation.

    Notes
    -----
    Executes the Q-learning agent in the environment with a 300s timeout.
    """
    # Create a new environment instance
    env = SubGymMarketsExecutionEnv_v0(parent_order_size=500, max_order_size=100)
    # Set the strategy for the environment
    env.strategy = strategy
    # Create a new Q-learning agent
    agent = QLearningAgent(env.action_space, strategy)
    # Initialize list to store rewards
    rewards = []
    # Initialize list to store execution costs
    execution_costs = []
    # Initialize list to store trade schedules
    trade_schedules = []
    # Initialize list to store market volumes
    market_volumes = []
    # Record the start time
    start_time = time.time()
    # Set timeout duration
    timeout = 300

    # Iterate over episodes
    for episode in range(episodes):
        # Reset the environment to start a new episode
        state = env.reset()
        # Check for timeout or invalid reset
        if state is None or time.time() - start_time > timeout:
            logger.error(f"Timeout or invalid reset after {time.time() - start_time:.1f} seconds")
            break
        # Initialize done flag
        done = False
        # Initialize episode reward
        episode_reward = 0.0
        # Reset execution cost for the episode
        env.custom_metrics_tracker.execution_cost = 0.0
        # Reset trade schedule for the episode
        env.custom_metrics_tracker.trade_schedule = []
        # Initialize list for episode volumes
        episode_volumes = []
        # Continue until episode is done or timeout
        while not done and time.time() - start_time < timeout:
            # Choose an action based on the current state
            action = agent.choose_action(state.flatten(), env)
            # Take a step in the environment
            next_state, reward, done, info = env.step(action)
            # Check for invalid next state
            if next_state is None:
                logger.error("Invalid next_state received")
                break
            # Update the agent's Q-table
            agent.learn(state.flatten(), action, reward, next_state.flatten(), env)
            # Accumulate episode reward
            episode_reward += reward
            # Append volume data
            episode_volumes.append(info["volume"])
            # Update state for next iteration
            state = next_state
        # Append episode reward to rewards list
        rewards.append(episode_reward)
        # Append episode execution cost to costs list
        execution_costs.append(info["execution_cost"])
        # Append episode trade schedule to schedules list
        trade_schedules.append(info["trade_schedule"])
        # Append episode volumes to volumes list
        market_volumes.append(episode_volumes)
        # Print progress every 2 episodes
        if episode % 2 == 0:
            elapsed = time.time() - start_time
            print(f"Episode {episode}, Strategy {strategy}, Reward: {np.mean(rewards[-2:]):.4f}, Cost: {np.mean(execution_costs[-2:]):.4f}, Time: {elapsed:.1f}s")
    # Return simulation results
    return rewards, execution_costs, trade_schedules, market_volumes

if __name__ == "__main__":
    # Define the list of strategies to test
    strategies = ["TWAP", "VWAP", "IS", "POV"]
    # Initialize dictionary to store results
    all_results = {}
    # Iterate over each strategy
    for strategy in strategies:
        # Print the start of training for the current strategy
        print(f"Training {strategy}...")
        # Run simulation and store results
        all_results[strategy] = run_simulation(strategy)

    import matplotlib.pyplot as plt

    # Function to compute theoretical trading trajectories
    def compute_theoretical_trajectory(strategy, parent_order_size=500, steps=12):
        """Compute the theoretical cumulative executed quantity for a strategy.
        
        Parameters
        ----------
        strategy : str
            The execution strategy (TWAP, VWAP, IS, or POV).
        parent_order_size : int
            The total order size (default: 500).
        steps : int
            Number of timesteps in the execution window (default: 12).

        Returns
        -------
        tuple
            (times, quantities) representing the theoretical trajectory.
        """
        # Generate time fractions for the execution window
        times = np.linspace(0, 1, steps)
        # Apply TWAP strategy: linear increase
        if strategy == "TWAP":
            quantities = times * parent_order_size
        # Apply VWAP strategy: stepwise increase over 10 steps
        elif strategy == "VWAP":
            step_sizes = np.linspace(0, 1, 10)[1:]  # Exclude 0 for 10 steps
            quantities = np.interp(times, np.linspace(0, 1, 10), step_sizes * parent_order_size)
        # Apply IS strategy: gradual increase with half max order size
        elif strategy == "IS":
            max_step = parent_order_size / (steps / 2)  # Rough estimate of half max per step
            quantities = np.minimum(times * parent_order_size, np.arange(steps) * max_step)
        # Apply POV strategy: proportional to 10% of volume, capped
        elif strategy == "POV":
            quantities = np.minimum(times * parent_order_size * 0.1 * 10, times * parent_order_size)
        # Return time and quantity arrays
        return times, quantities

    # Generate four separate plots for each strategy
    for strategy in strategies:
        rewards, execution_costs, trade_schedules, market_volumes = all_results[strategy]
        plt.figure(figsize=(10, 6))

        # Plot learned schedule
        schedule = trade_schedules[-1]
        times, quantities = zip(*schedule) if schedule else ([], [])
        if times:
            plt.plot(times, quantities, 'b-', label=f"{strategy} Learned Trades")

        # Plot theoretical schedule
        theo_times, theo_quantities = compute_theoretical_trajectory(strategy)
        plt.plot(theo_times, theo_quantities, 'r--', label=f"{strategy} Theoretical Trades")

        plt.xlabel('Time (Fraction of Execution Window)')
        plt.ylabel('Cumulative Executed Quantity')
        plt.title(f'Trade Schedule for {strategy}')
        plt.legend()
        plt.grid(True)
        plt.show()
    
    # Execution Cost Comparison
    plt.figure(figsize=(10, 6))
    costs = [all_results[strategy][1] for strategy in strategies]
    plt.boxplot(costs, labels=strategies)
    plt.xlabel('Strategy')
    plt.ylabel('Execution Cost (Normalized)')
    plt.title('Execution Cost Comparison')
    plt.grid(True)
    plt.show()
    
    # Reward Plot
    plt.figure(figsize=(10, 6))
    for strategy, (rewards, _, _, _) in all_results.items():
        plt.plot(np.arange(len(rewards)), rewards, label=strategy)
    plt.xlabel('Episodes')
    plt.ylabel('Reward')
    plt.title('Reward Curves')
    plt.legend()
    plt.grid(True)
    plt.show()


2025-06-20 15:39:10,478 - DEBUG - Num max steps per episode: 14.0
2025-06-20 15:39:10,480 - DEBUG - SparseMeanRevertingOracle computing initial fundamental value for ABM
2025-06-20 15:39:10,481 - DEBUG - SparseMeanRevertingOracle initialized for symbols {'ABM': {'r_bar': 100000, 'kappa': 1.67e-16, 'sigma_s': 0, 'fund_vol': 5e-05, 'megashock_lambda_a': 2.77778e-18, 'megashock_mean': 1000, 'megashock_var': 50000, 'random_state': RandomState(MT19937) at 0x168C42540}}
2025-06-20 15:39:10,481 - DEBUG - SparseMeanRevertingOracle initialization took 0:00:00.000678
2025-06-20 15:39:10,497 - DEBUG - Detected 1 ABIDES-gym agents
2025-06-20 15:39:10,498 - DEBUG - Kernel initialized
2025-06-20 15:39:10,498 - DEBUG - Kernel started
2025-06-20 15:39:10,498 - DEBUG - Simulation started!
2025-06-20 15:39:10,499 - DEBUG - --- Agent.kernel_initializing() ---
2025-06-20 15:39:10,499 - DEBUG - EXCHANGE_AGENT exists!
2025-06-20 15:39:10,499 - DEBUG - Oracle: client requested ABM at market open: 16125174000

Training TWAP...


2025-06-20 15:39:10,679 - DEBUG - Agent 105 (Value Agent 105) requesting kernel wakeup at time 2021-02-05 00:00:00
2025-06-20 15:39:10,680 - DEBUG - Agent 106 requested agent of type Agent.ExchangeAgent.  Given Agent ID: 0
2025-06-20 15:39:10,680 - DEBUG - Agent 106 (Value Agent 106) requesting kernel wakeup at time 2021-02-05 00:00:00
2025-06-20 15:39:10,680 - DEBUG - Agent 107 requested agent of type Agent.ExchangeAgent.  Given Agent ID: 0
2025-06-20 15:39:10,681 - DEBUG - Agent 107 (Value Agent 107) requesting kernel wakeup at time 2021-02-05 00:00:00
2025-06-20 15:39:10,681 - DEBUG - Agent 108 requested agent of type Agent.ExchangeAgent.  Given Agent ID: 0
2025-06-20 15:39:10,682 - DEBUG - Agent 108 (ADAPTIVE_POV_MARKET_MAKER_AGENT_108) requesting kernel wakeup at time 2021-02-05 00:00:00
2025-06-20 15:39:10,682 - DEBUG - Agent 109 requested agent of type Agent.ExchangeAgent.  Given Agent ID: 0
2025-06-20 15:39:10,682 - DEBUG - Agent 109 (ADAPTIVE_POV_MARKET_MAKER_AGENT_109) reques

TypeError: object of type 'int' has no len()


## Definition of the Four Strategies
The experiment evaluates four algorithmic order execution strategies, each designed to optimize the execution of a parent order (500 shares) within a 1-minute execution window. Their definitions and expected trade behaviors are as follows:

- **TWAP (Time-Weighted Average Price)**:
  - **Definition**: Distributes the parent order uniformly over the execution window based on time, aiming to achieve the time-weighted average price. The order size is proportional to the fraction of time elapsed (`time_pct`).
  - **Expected Plot**: A smooth, linear increase in cumulative executed quantity over time, starting from 0 and reaching approximately 500 shares by the end of the window (time fraction 1.0), assuming consistent market liquidity.

- **VWAP (Volume-Weighted Average Price)**:
  - **Definition**: Allocates the order based on a linear approximation of market volume, targeting the volume-weighted average price. The order size increases in steps proportional to the elapsed time and a volume profile (simplified to 10 equal steps).
  - **Expected Plot**: A stepwise increase in cumulative executed quantity, with jumps at regular intervals (e.g., every 0.1 time fraction), reaching 500 shares by the end, reflecting volume-based pacing.

- **IS (Implementation Shortfall)**:
  - **Definition**: Minimizes the difference between the execution price and the arrival price, using half the maximum order size per trade to balance speed and cost. It adjusts based on the difference between executed and target quantity.
  - **Expected Plot**: A gradual increase in cumulative executed quantity, potentially with irregular steps due to cautious trading, aiming for 500 shares but possibly under-executing if liquidity is low.

- **POV (Percentage of Volume)**:
  - **Definition**: Executes orders proportional to 10% of the current market volume, adapting to real-time liquidity. The order size depends on the observed volume at each timestep.
  - **Expected Plot**: A variable increase in cumulative executed quantity, reflecting market volume fluctuations, with potential plateaus or rapid rises depending on volume availability, targeting 500 shares.
